# Análisis de Sentimiento con una Red LSTM usando Keras

## Objetivo

En esta actividad vas a construir un modelo de red neuronal recurrente (RNN), específicamente una LSTM, usando la API Keras de TensorFlow. El modelo va a leer frases en español y clasificar su sentimiento como positivo o negativo.

### ¿Qué vamos a lograr?

- Entender cómo las redes recurrentes procesan secuencias de palabras
- Implementar una LSTM que recuerda el contexto de la frase
- Usar embeddings de palabras para representar el significado
- Observar cómo las LSTM superan las limitaciones de bag-of-words

### ¿Qué es una LSTM?

LSTM (Long Short-Term Memory) es un tipo especial de red neuronal recurrente diseñada para:
- **Procesar secuencias**: Lee las palabras en orden, una después de la otra
- **Mantener memoria**: Recuerda información importante de palabras anteriores
- **Olvidar información irrelevante**: Decide qué información del pasado mantener y qué descartar

A diferencia de las MLP que vimos antes, las LSTM **sí consideran el orden** de las palabras, lo cual es fundamental para entender el lenguaje.

## 1. Preparación del entorno

Importamos las librerías necesarias, incluyendo herramientas de Keras para procesamiento de secuencias.

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(f"TensorFlow/Keras versión: {keras.__version__}")

TensorFlow/Keras versión: 3.10.0


## 2. Datos de entrenamiento

Vamos a usar las mismas frases que en la actividad anterior, pero ahora las vamos a procesar como **secuencias de palabras**, no como bolsa de palabras.

Esta diferencia es fundamental: la LSTM va a poder aprovechar el orden en que aparecen las palabras.

In [2]:
frases = [
    "La verdad, este lugar está bárbaro. Muy recomendable",
    "Una porquería de servicio, nunca más vuelvo",
    "Me encantó la comida, aunque la música estaba muy fuerte",
    "El envío fue lento y el producto llegó dañado. Qué desastre",
    "Todo excelente. Atención de diez",
    "Qué estafa, me arrepiento de haber comprado",
    "Muy conforme con el resultado final",
    "No me gustó para nada la experiencia",
    "Superó mis expectativas, gracias",
    "No lo recomiendo, mala calidad"
]

etiquetas = np.array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0])

print(f"Total de frases: {len(frases)}")
print(f"Balance: {sum(etiquetas)} positivas, {len(etiquetas) - sum(etiquetas)} negativas\n")
print("Ejemplos:")
for i in range(3):
    sentimiento = "Positivo" if etiquetas[i] == 1 else "Negativo"
    print(f"  {i+1}. '{frases[i][:50]}...' → {sentimiento}")

Total de frases: 10
Balance: 5 positivas, 5 negativas

Ejemplos:
  1. 'La verdad, este lugar está bárbaro. Muy recomendab...' → Positivo
  2. 'Una porquería de servicio, nunca más vuelvo...' → Negativo
  3. 'Me encantó la comida, aunque la música estaba muy ...' → Positivo


## 3. Tokenización y construcción del vocabulario

Con Keras, vamos a convertir las frases en secuencias de números, donde cada número representa una palabra del vocabulario.

### Diferencia clave con bag-of-words:

**Bag-of-words:**
- "Me gusta" → [1, 0, 1, 0, 0] (solo presencia/ausencia)

**Secuencia:**
- "Me gusta" → [5, 12] (orden preservado, cada palabra tiene un ID)

In [3]:
# Tokenización: convierte palabras a números
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(frases)

# Mostramos el vocabulario construido
vocab_size = len(tokenizer.word_index) + 1  # +1 por el índice 0
print(f"Tamaño del vocabulario: {vocab_size} palabras únicas\n")
print("Primeras 15 palabras del vocabulario:")
for palabra, idx in list(tokenizer.word_index.items())[:15]:
    print(f"  '{palabra}' → {idx}")

# Convertimos frases a secuencias numéricas
secuencias = tokenizer.texts_to_sequences(frases)

print(f"\nEjemplo de conversión:")
print(f"Frase original: '{frases[0]}'")
print(f"Secuencia numérica: {secuencias[0]}")

Tamaño del vocabulario: 59 palabras únicas

Primeras 15 palabras del vocabulario:
  '<OOV>' → 1
  'la' → 2
  'muy' → 3
  'de' → 4
  'me' → 5
  'el' → 6
  'qué' → 7
  'no' → 8
  'verdad' → 9
  'este' → 10
  'lugar' → 11
  'está' → 12
  'bárbaro' → 13
  'recomendable' → 14
  'una' → 15

Ejemplo de conversión:
Frase original: 'La verdad, este lugar está bárbaro. Muy recomendable'
Secuencia numérica: [2, 9, 10, 11, 12, 13, 3, 14]


## 4. Padding: estandarizando la longitud de las secuencias

Las redes neuronales necesitan entradas de tamaño fijo, pero nuestras frases tienen longitudes diferentes.

**Solución: Padding**
- Rellenamos las secuencias cortas con ceros al final
- Todas las secuencias terminan con la misma longitud

Ejemplo:
- `[5, 12]` → `[5, 12, 0, 0, 0]` (padding='post')

In [4]:
# Calculamos la longitud máxima
maxlen = max(len(seq) for seq in secuencias)
print(f"Longitud de la frase más larga: {maxlen} palabras\n")

# Aplicamos padding
X = pad_sequences(secuencias, maxlen=maxlen, padding='post')
y = np.array(etiquetas)

print(f"Forma de X después del padding: {X.shape}")
print(f"  {X.shape[0]} frases × {X.shape[1]} posiciones\n")

print("Ejemplo de secuencia con padding:")
print(f"Frase: '{frases[0]}'")
print(f"Secuencia: {X[0]}")
print(f"Nota: Los ceros al final son padding (relleno)")

Longitud de la frase más larga: 11 palabras

Forma de X después del padding: (10, 11)
  10 frases × 11 posiciones

Ejemplo de secuencia con padding:
Frase: 'La verdad, este lugar está bárbaro. Muy recomendable'
Secuencia: [ 2  9 10 11 12 13  3 14  0  0  0]
Nota: Los ceros al final son padding (relleno)


## 5. Definición del modelo LSTM

Vamos a construir una red con tres componentes clave:

### 1. Capa de Embedding
Convierte cada palabra (número) en un vector denso de dimensión fija. Estos vectores se aprenden durante el entrenamiento y capturan similitudes semánticas.

**Ejemplo conceptual:**
- "excelente" → [0.8, 0.9, -0.1, 0.7, ...]
- "bueno" → [0.7, 0.8, -0.2, 0.6, ...] (vector similar)
- "malo" → [-0.7, -0.8, 0.2, -0.6, ...] (vector opuesto)

### 2. Capa LSTM
Procesa la secuencia de embeddings manteniendo memoria del contexto.

### 3. Capa Dense (salida)
Clasifica el sentimiento basándose en la representación aprendida por la LSTM.

In [5]:
# Parámetros del modelo
embedding_dim = 16  # Dimensión de los vectores de embeddings
lstm_units = 32     # Número de unidades en la capa LSTM

# Construcción del modelo
modelo = Sequential([
    # Capa 1: Embedding - convierte palabras a vectores densos
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen),

    # Capa 2: LSTM - procesa la secuencia con memoria
    LSTM(units=lstm_units),

    # Capa 3: Dense - clasificación final
    Dense(1, activation='sigmoid')
])

# Compilación del modelo
modelo.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Construir el modelo explícitamente
modelo.build(input_shape=X.shape)

print("Arquitectura del modelo:")
modelo.summary()

print(f"\nParámetros totales: {modelo.count_params():,}")

Arquitectura del modelo:


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (10, 11, 16)           │           944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (10, 32)               │         6,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (10, 1)                │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,249 (28.32 KB)

 Trainable params: 7,249 (28.32 KB)

 Non-trainable params: 0 (0.00 B)


Parámetros totales: 7,249


## 6. Entrenamiento

Entrenamos el modelo por varias épocas. La LSTM va a aprender:
- Qué palabras son importantes para el sentimiento
- Cómo el orden afecta el significado
- Qué patrones secuenciales indican positivo o negativo

In [8]:
print("="*60)
print("INICIANDO ENTRENAMIENTO")
print("="*60)
print(f"Épocas: 20")
print(f"Batch size: 2\n")

# Entrenamiento
history = modelo.fit(
    X, y,
    epochs=30,
    batch_size=4,
    verbose=1
)

print("\n" + "="*60)
print("ENTRENAMIENTO FINALIZADO")
print("="*60)

INICIANDO ENTRENAMIENTO
Épocas: 20
Batch size: 2

Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 1.0000 - loss: 0.0280
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.0267
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0242
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0218
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.0204 
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.0196
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0188
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0178
Epoch 9/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 1.0000 - loss: 0.0172
Epoch 10/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0160
Epoch 11/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.0153
Epoch 12/30
3/3 ━━━━━━━━━━━━━━━━━━

## 7. Análisis del entrenamiento

Observá cómo evolucionan la pérdida (loss) y la precisión (accuracy) durante el entrenamiento.

- **Loss decrece**: El modelo comete menos errores
- **Accuracy aumenta**: Más predicciones correctas

Si la accuracy llega a 1.0, significa que el modelo clasificó perfectamente todos los ejemplos de entrenamiento.

## 8. Evaluación con frases nuevas

Ahora vamos a probar el modelo con frases que no vio durante el entrenamiento. Esta es la verdadera prueba de si aprendió patrones generalizables.

In [14]:
frases_nuevas = [
    "Muy buena atención, quedé encantado",
    "Horrible experiencia, no vuelvo más",
    "Todo excelente, gracias por la atención",
    "Me arrepiento completamente, fue un desastre",
    "horrible atención"
]

print("="*60)
print("EVALUACIÓN EN FRASES NUEVAS")
print("="*60)

# Tokenizamos y aplicamos padding
secuencias_nuevas = tokenizer.texts_to_sequences(frases_nuevas)
X_nuevo = pad_sequences(secuencias_nuevas, maxlen=maxlen, padding='post')

# Predicción
predicciones = modelo.predict(X_nuevo, verbose=0)

# Mostrar resultados
for i, (frase, pred) in enumerate(zip(frases_nuevas, predicciones), 1):
    probabilidad = pred[0]
    clase = "Positivo" if probabilidad >= 0.5 else "Negativo"

    print(f"\nFrase {i}: '{frase}'")
    print(f"  Predicción: {clase} (probabilidad: {probabilidad:.2f})")

    # Indicador de confianza
    if probabilidad >= 0.8 or probabilidad <= 0.2:
        print(f"  Confianza: Alta")
    elif probabilidad >= 0.6 or probabilidad <= 0.4:
        print(f"  Confianza: Media")
    else:
        print(f"  Confianza: Baja (ambiguo)")

print("\n" + "="*60)

EVALUACIÓN EN FRASES NUEVAS

Frase 1: 'Muy buena atención, quedé encantado'
  Predicción: Positivo (probabilidad: 0.99)
  Confianza: Alta

Frase 2: 'Horrible experiencia, no vuelvo más'
  Predicción: Negativo (probabilidad: 0.01)
  Confianza: Alta

Frase 3: 'Todo excelente, gracias por la atención'
  Predicción: Positivo (probabilidad: 0.99)
  Confianza: Alta

Frase 4: 'Me arrepiento completamente, fue un desastre'
  Predicción: Negativo (probabilidad: 0.01)
  Confianza: Alta

Frase 5: 'horrible atención'
  Predicción: Positivo (probabilidad: 0.99)
  Confianza: Alta



## 9. Comparación con enfoques anteriores

Recapitulemos lo que mejoramos con cada modelo:

### Perceptrón simple:
- ✗ No considera orden de palabras
- ✗ Representación binaria (0/1)
- ✗ Modelo lineal
- ✓ Muy simple de entender

### MLP (Red Multicapa):
- ✗ No considera orden de palabras
- ✗ Representación binaria (0/1)
- ✓ Puede aprender patrones no lineales
- ✓ Mejor capacidad de generalización

### LSTM:
- ✓ **Considera el orden de las palabras**
- ✓ **Embeddings aprendidos** (vectores densos)
- ✓ **Memoria del contexto** (puede recordar palabras anteriores)
- ✓ Puede aprender patrones no lineales complejos

La LSTM es un avance significativo porque finalmente podemos procesar el lenguaje como una secuencia, no como una bolsa desordenada de palabras.

## 10. Reflexión final

### ¿Qué aprendimos?

1. **Procesamiento de secuencias**: Las LSTM pueden leer frases palabra por palabra, manteniendo memoria del contexto.

2. **Embeddings de palabras**: En lugar de vectores binarios (0/1), cada palabra se representa con un vector denso que captura su significado.

3. **Orden importa**: "No me gusta" y "Me gusta, no" ahora se procesan diferente (antes eran idénticos con bag-of-words).

4. **Tokenización automática**: Keras construye el vocabulario automáticamente y maneja palabras desconocidas con `<OOV>`.

### Ventajas sobre MLP con bag-of-words:

- Captura el orden de las palabras
- Aprende representaciones semánticas (embeddings)
- Puede detectar patrones secuenciales
- Mejor manejo de frases largas

### Limitaciones que aún persisten:

1. **Procesamiento secuencial**: La LSTM lee de izquierda a derecha, puede "olvidar" información del principio en frases muy largas

2. **No puede mirar hacia adelante**: Al procesar una palabra, no sabe qué viene después

3. **Dataset pequeño**: Con solo 10 ejemplos, los embeddings no se entrenan bien

4. **Vocabulario limitado**: Solo conoce las palabras que aparecieron en el entrenamiento

### ¿Qué sigue?

En la próxima actividad vamos a ver cómo los **modelos preentrenados** como BETO (BERT en español) resuelven muchas de estas limitaciones:

- Ya fueron entrenados con millones de textos
- Tienen embeddings muy ricos
- Usan arquitectura Transformer (no secuencial, con **atención**)
- Pueden hacer análisis de sentimiento sin necesidad de entrenar desde cero

Esto nos va a llevar al concepto de **transfer learning**, que revolucionó el NLP y es la base de los LLMs modernos como GPT.